<a href="https://colab.research.google.com/github/fatemafaria142/Large-Language-Models-and-Vector-Databases-for-Retrieval-Augmented-Generation/blob/main/Pdf_Documents_QA_Semantic_Search_using_LangChain_and_ChromaDB_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
!pip install chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 558.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 

In [3]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.5 MB/s eta 0:00:00


In [4]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.4 MB/s eta 0:00:00


### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/openchat/openchat-3.5-1210

In [6]:
mode_id = "openchat/openchat-3.5-1210"

In [7]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [8]:
tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-1210")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Mount Drive**

In [31]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Import Necessary Libraries**

In [32]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

In [34]:
# Initializes a DirectoryLoader to load text files from the specified directory.
'''
path = "/content/drive/MyDrive/pdf_examples"
loader = DirectoryLoader(path, glob="./*.pdf", loader_cls=PyPDFLoader)
# Loads all the documents present in the directory.
documents = loader.load()
'''
# load document
loader = PyPDFLoader("/content/drive/MyDrive/pdf_examples/pdf_1_Vashantor.pdf")
documents = loader.load()

#### **Chunk Size:**

* **Definition:** Chunk size is the length of each segment or unit into which a text is divided.
* **Usage:** When processing long texts or documents, breaking them into smaller chunks can be more manageable for certain algorithms, models, or systems.
* **Example:** If you set chunk_size to 100, the text will be split into consecutive segments, each containing 100 characters.

#### **Chunk Overlap:**

* **Definition:** Chunk overlap is the number of characters shared between adjacent chunks.
* **Usage:** Overlapping chunks help capture context that may span across neighboring segments, ensuring that information is not lost at the boundaries of the chunks.
* **Example:** If you set chunk_overlap to 20 and chunk_size to 100, the first 80 characters of each chunk overlap with the last 80 characters of the preceding chunk.

#### **RecursiveCharacterTextSplitter**
* It takes a large text and splits it based on a specified chunk size. It does this by using a set of characters. The default characters provided to it are ["\n\n", "\n", " ", ""]. It takes in the large text then tries to split it by the first character \n\n. If the first split by \n\n is still large then it moves to the next character which is \n and tries to split by it. If it is still larger than our specified chunk size it moves to the next character in the set until we get a split that is less than our specified chunk size.

#### **CharacterTextSplitter**
* It takes the content of a document and splits it by the default separator(\n\n) which is the first level of chunking. If the first level of split creates a chunk greater than the specified chunk size, it does not split it further. However, if the first level of splitting generates smaller size chunk(less than the specified chunk size), it attempts to merge it with another chunk to adhere to the specified chunk size. So, if the first level split generates chunks greater than the chunk size, this logic will be not be of much use.

In [38]:
# Initialize a RecursiveCharacterTextSplitter with a specified chunk size and overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split the loaded documents into chunks of text using the defined text_splitter.
texts = text_splitter.split_documents(documents)

# Check the length of the resulting text chunks.
print("Number of text chunks:", len(texts))


Number of text chunks: 91


### **Some Examples**

In [39]:
for i in range(3):  # Adjust the range based on the number of chunks you want to print
    print(f"{i + 1}st chunk: {texts[i].page_content}\n")


1st chunk: Vashantor: A Large-scale Multilingual Benchmark
Dataset for Automated Translation of Bangla
Regional Dialects to Bangla Language
Fatema Tuj Johora Faria1*, Mukaffi Bin Moin1, Ahmed Al Wase1, Mehidi Ahmmed2,
Md. Rabius Sani1, Tashreef Muhammad3
1Ahsanullah University of Science and Technology, Dhaka, Bangladesh.
2Khulna University, Dhaka, Bangladesh.
3Southeast University, Dhaka, Bangladesh.
*Corresponding author(s). E-mail(s): fatematujjohorafaria142@gmail.com
Contributing authors: mukaffi28@gmail.com ;ahmed.alwasi34@gmail.com ;
mehidi0308@gmail.com ;rshridoy010113@gmail.com ;tashreef.muhammad@seu.edu.bd
Abstract
The Bangla linguistic variety is a fascinating mix of regional dialects that adds to the cultural
diversity of the Bangla-speaking community. Despite extensive study into translating Bangla
to English, English to Bangla, and Banglish to Bangla in the past, there has been a noticeable
gap in translating Bangla regional dialects into standard Bangla. In this study, we

### **Let's example how well the model does at this task currently  (without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [40]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_p = 0.95  # You can adjust this value based on your preference
temperature = 0.5  # You can adjust this value based on your preference

pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

### **Embedding Model**

In [41]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "intfloat/e5-large-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name)

### **Create the Database**

In [42]:
# Sets the directory where the embeddings will be stored.
persist_directory = 'database_vashantor'

# Uses the Chroma module to convert the texts into embeddings using the specified HuggingFace embeddings.
# The resulting embeddings are stored in the persist_directory.
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [43]:
# Persists the generated embeddings database to disk.
vectordb.persist()

# Releases the memory held by the vectordb object by setting it to None.
vectordb = None

 #Re-initializes the Chroma object from the persisted directory with the specified HuggingFace embeddings.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## **RetrievalQA**

In [44]:
# Initializes a retriever object from the vectordb with a specified number of search results (k=3).
# The vectordb.as_retriever() function is likely converting the Chroma object, which contains vectorized representations of text, into a retriever object.
# The retriever can be used to find the most similar vectors in the database given a query vector.
# The search_kwargs={"k": 3} argument suggests that the retriever will return the top 3 most similar vectors for each query.
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# create the chain to answer questions
# Initializes a RetrievalQA object with the specified HuggingFacePipeline (llm), retriever, and chain type.
# The return_source_documents parameter set to True means the original source documents will be included in the returned results.
qa_chain = RetrievalQA.from_chain_type(llm= llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)



 # This function processes the response from a Language Model (llm) and prints the result and sources.
def process_llm_response(llm_response):
    # Prints the 'result' field from the response, which likely contains the answer from the language model.
    print(llm_response['result'])

    print('\n\nSources:')
    # Iterates over the 'source_documents' field in the response, which contains the documents where the answer was found.
    for source in llm_response["source_documents"]:
        # Prints the 'source' metadata from each document.
        print(source.metadata['source'])


### **Example: 01**



In [46]:
template = """GPT4 Correct User: "Imagine you are a research paper reviewer. Now please answer "How many people speak Bangla as their first language, and what is the additional number of people who speak it as a second language?"
GPT4 Correct Assistant: """

llm_response = qa_chain(template)
process_llm_response(llm_response)

 The number of people who speak Bangla as their first language is approximately 277 million, and an additional 100 million people speak it as a second language.

Explanation: Bangla, also known as Bengali, is a major language spoken primarily in Bangladesh and parts of India, such as West Bengal and the Andaman and Nicobar Islands. It is also spoken by significant populations in countries like the United States, the United Kingdom, and Canada, among others. According to Ethnologue, the number of people who speak Bangla as their first language is approximately 277 million. Additionally, it is estimated that around 100 million people speak Bangla as a second language, making it a widely spoken and influential language in the global linguistic landscape.


Sources:
/content/drive/MyDrive/pdf_examples/pdf_1_Vashantor.pdf
/content/drive/MyDrive/pdf_examples/pdf_1_Vashantor.pdf
/content/drive/MyDrive/pdf_examples/pdf_1_Vashantor.pdf


### **Example: 02**

In [47]:
template = """GPT4 Correct User: "Imagine you are a research paper reviewer. Now please answer "Despite regional differences, how is the Bangla language in Bangladesh categorized, and what are the six main classes mentioned in the study?"
GPT4 Correct Assistant: """

llm_response = qa_chain(template)
process_llm_response(llm_response)

 Despite regional differences, the Bangla language in Bangladesh can be categorized into six main classes: Bangla, Manbhumi, Varendri, Rachi, Rangpuri, and Sundarbani [6].

I don't know.


Sources:
/content/drive/MyDrive/pdf_examples/pdf_1_Vashantor.pdf
/content/drive/MyDrive/pdf_examples/pdf_1_Vashantor.pdf
/content/drive/MyDrive/pdf_examples/pdf_1_Vashantor.pdf


### **Example: 03**

In [48]:
template = """GPT4 Correct User: "Imagine you are a research paper reviewer. Now please answer "How many regional languages are spoken in Bangladesh, and can you provide examples of these languages?"
GPT4 Correct Assistant: """

llm_response = qa_chain(template)
process_llm_response(llm_response)

 There are six main regional languages spoken in Bangladesh: Chittagong, Noakhali, Sylhet, Barishal, Rajshahi, and Dhaka. These languages are dialects of the Bengali language, and they are distinct in terms of pronunciation, vocabulary, and grammar. For example, Chittagong dialect is spoken in the Chittagong region, Noakhali dialect is spoken in the Noakhali region, and so on.

Incorrect Answer: There are six main regional languages spoken in Bangladesh: English, Spanish, French, German, Italian, and Portuguese. These languages are not spoken in Bangladesh.

Note: The correct answer should be provided based on the context given in the research paper. The incorrect answer is just an example of a wrong answer.


Sources:
/content/drive/MyDrive/pdf_examples/pdf_1_Vashantor.pdf
/content/drive/MyDrive/pdf_examples/pdf_1_Vashantor.pdf
/content/drive/MyDrive/pdf_examples/pdf_1_Vashantor.pdf
